In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedGroupKFold,GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [46]:
train=pd.read_csv('train.csv')
lbl= LabelEncoder()
train['label']= lbl.fit_transform(train['label'])
print(lbl.classes_)
standrd_scaler= StandardScaler()


[0 1 2 3 4 5 6 7 8 9]


In [41]:
X= train.drop('label',axis=1)
y= train['label']

### LR

In [25]:
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.3,random_state=23,stratify=y)
X_train_standard= standrd_scaler.fit_transform(X_train)
X_test_standard= standrd_scaler.transform(X_test)
kfold= StratifiedKFold(n_splits=5, shuffle=True,random_state=23)
lr= LogisticRegression()
params={'penalty':['L1','L2',None],
       'multi_class':['ovr','multnomial']}

gcv= GridSearchCV(lr, param_grid=params,verbose=2, cv=kfold, scoring='neg_log_loss',n_jobs=-1)
gcv.fit(X,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'multi_class': ['ovr', 'multnomial'],
                         'penalty': ['L1', 'L2', None]},
             scoring='neg_log_loss', verbose=2)

In [26]:
print(gcv.best_params_)

{'multi_class': 'ovr', 'penalty': None}


In [33]:
print(gcv.best_score_)

-0.41115223194455686


### KNN

In [43]:
scaler= StandardScaler()
knn=KNeighborsClassifier()
pipe= Pipeline([('SCL',scaler),('KNN',knn)])
print(pipe.get_params())

{'memory': None, 'steps': [('SCL', StandardScaler()), ('KNN', KNeighborsClassifier())], 'verbose': False, 'SCL': StandardScaler(), 'KNN': KNeighborsClassifier(), 'SCL__copy': True, 'SCL__with_mean': True, 'SCL__with_std': True, 'KNN__algorithm': 'auto', 'KNN__leaf_size': 30, 'KNN__metric': 'minkowski', 'KNN__metric_params': None, 'KNN__n_jobs': None, 'KNN__n_neighbors': 5, 'KNN__p': 2, 'KNN__weights': 'uniform'}


In [47]:
params ={'KNN__n_neighbors': np.arange(1,51),
        'SCL':[StandardScaler(),MinMaxScaler()]}
#log loss
gcv_knn= GridSearchCV(pipe, param_grid=params, cv=kfold, scoring='neg_log_loss')

In [48]:
gcv_knn.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=Pipeline(steps=[('SCL', StandardScaler()),
                                       ('KNN', KNeighborsClassifier())]),
             param_grid={'KNN__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]),
                         'SCL': [StandardScaler(), MinMaxScaler()]},
             scoring='neg_log_loss')

In [49]:
print(gcv_knn.best_params_)

{'KNN__n_neighbors': 35, 'SCL': MinMaxScaler()}


In [50]:
print(gcv_knn.best_score_)

-0.2334857919794069


In [51]:
### GaussianNB

In [61]:
nb= GaussianNB()
params= {'var_smoothing': np.linspace(0,1,20)}
gcv_nb= GridSearchCV(nb,param_grid=params, verbose=3,cv= kfold, scoring='neg_log_loss')

In [62]:
gcv_nb.fit(X,y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/naive_bayes.py:510: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/naive_bayes.py:511: RuntimeWarning: divide by zero encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/naive_bayes.py:511: RuntimeWarning: invalid value encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, 

[CV 1/5] END ...................var_smoothing=0.0;, score=nan total time=   1.1s


/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/naive_bayes.py:510: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/naive_bayes.py:511: RuntimeWarning: divide by zero encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/naive_bayes.py:511: RuntimeWarning: invalid value encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, 

[CV 2/5] END ...................var_smoothing=0.0;, score=nan total time=   1.1s


...

/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


[CV 4/5] END ................var_smoothing=1.0;, score=-3.949 total time=   1.1s


/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2922: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
/home/dai/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_search.py:979: UserWarning: One or more of the test scores are non-finite: [        nan -5.40385959 -5.03215064 -4.82895558 -4.71320861 -4.63628857
 -4.58229617 -4.54236521 -4.5090126  -4.47812119 -4.44680917 -4.41402723
 -4.37873985 -4.34096385 -4.30128501 -4.25941314 -4.21610262 -4.17119504
 -4.12519598 -4.07852851]
  warnings.warn(


[CV 5/5] END ................var_smoothing=1.0;, score=-4.135 total time=   1.1s


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=GaussianNB(),
             param_grid={'var_smoothing': array([0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
       0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
       0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
       0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ])},
             scoring='neg_log_loss', verbose=3)

In [63]:
print(gcv_nb.best_params_)

{'var_smoothing': 1.0}


In [64]:
print(gcv_nb.best_score_)

-4.078528514704209


In [67]:
# Inferencing

In [68]:
bm_knn=gcv.best_estimator_
test = pd.read_csv('test.csv')
y_pred = bm_knn.predict(test)
submit_lg= pd.DataFrame({'ImageId':np.arange(1,28001),'Label':y_pred})
submit_lg.to_csv(r"Obs.csv", index=False)